In [ ]:
import os
import shutil

import jpype.imports

from glob import glob
from jpype import startJVM, shutdownJVM, java, addClassPath, JClass, JInt

from datagen_image import run_test

startJVM(classpath='bin', convertStrings=False)

Select a model from:
- LeNet-7 (le7)
- LeNet-9 (le9)
- ResNet-18 (res18)
- ResNet-152 (res152)
- EfficientNetB0 (efficientb0)

In [ ]:
mod_name = 'le9'

Type the file path to your newspaper page

In [ ]:
path_to_newspaper_page = 'sn83035366_1836-02-12_ed-1_seq-4.jpg'

Are you using a fine tuned model?

In [ ]:
finetuned = False

The following script attempts to detect poems in the newspaper page

In [ ]:
directory_path = 'Output_Snippets'
if not os.path.exists(directory_path):
    os.mkdir(directory_path)
    
try:
    _seg = JClass('execute/RunPageSegmentation')
    obj = _seg.run_seg(path_to_newspaper_page)
#     print(obj)
except Exception as err:
    print(f"Exception: {err}")

flist = [y for x in os.walk(directory_path) 
         for y in glob(os.path.join(x[0], '*.jpg'))]
        
base_path = ''
nClass = 2
nEpoch = 100

if finetuned:
    model_path = 'pretrained_model/' + mod_name + '_aida17k_192_128_weights_finetune.h5'
else:
    model_path = 'pretrained_model/' + mod_name + '_aida17k_192_128_weights.h5'

test_preds = run_test(flist,
                      model_path, 
                      base_path,
                      nClass, 
                      mod_name)

try:
    shutil.rmtree(directory_path)
except OSError:
    print("Error occurred while deleting intermediate files.")

print("Poem is detected on the page?")
print(1 in test_preds)